In [176]:
import os
import sys
import pathlib
import click
import yaml
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

In [177]:
import time
import glob
import curses
import numpy as np
#sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname(__file__))))
from eval.utils.get_index_faiss import get_index
from eval.utils.print_table import PrintTable

In [178]:
from eval.eval_faiss import eval_faiss

In [179]:
def load_config(config_fname):
    config_filepath = './config/' + config_fname + '.yaml'
    if os.path.exists(config_filepath):
        print(f'cli: Configuration from {config_filepath}')
    else:
        sys.exit(f'cli: ERROR! Configuration file {config_filepath} is missing!!')

    with open(config_filepath, 'r') as f:
        cfg = yaml.safe_load(f)
    return cfg

In [180]:
def load_memmap_data(source_dir,
                     fname,
                     append_extra_length=None,
                     shape_only=False,
                     display=True):
    """
    Load data and datashape from the file path.

    • Get shape from [source_dir/.npy}.
    • Load memmap data from [source_dir/fname.mm].

    Parameters
    ----------
    source_dir : (str)
    fname : (str)
        File name except extension.
    append_empty_length : None or (int)
        Length to appened empty vector when loading memmap. If activate, the
        file will be opened as 'r+' mode.
    shape_only : (bool), optional
        Return only shape. The default is False.
    display : (bool), optional
        The default is True.

    Returns
    -------
    (data, data_shape)

    """
    path_shape = source_dir + fname + '_shape.npy'
    path_data = source_dir + fname + '.mm'
    data_shape = np.load(path_shape)
    if shape_only:
        return data_shape

    if append_extra_length:
        data_shape[0] += append_extra_length
        data = np.memmap(path_data, dtype='float32', mode='r+',
                         shape=(data_shape[0], data_shape[1]))
    else:
        data = np.memmap(path_data, dtype='float32', mode='r',
                         shape=(data_shape[0], data_shape[1]))
    if display:
        print(f'Load {data_shape[0]:,} items from \033[32m{path_data}\033[0m.')
    return data, data_shape

In [181]:
checkpoint_name:str = "Checks_test_generate"   # string
checkpoint_index:int = 100  # int
config:str = "default"       # string 'default'
index_type:str = 'IVFPQ'  # {'L2', 'IVF', 'IVFPQ', " + "'IVFPQ-RR', 'IVFPQ-ONDISK', HNSW'}"
test_seq_len:str =  '11'   # string '1 3 5 9 11 19' segundos 
test_ids:str = "icassp"
nogpu:bool = False
emb_dummy_dir:str = None
max_train:int = 1e7
k_probe:int = 20

In [182]:
cfg = load_config(config)
emb_dir = cfg['DIR']['OUTPUT_ROOT_DIR'] + checkpoint_name + '/' + \
    str(checkpoint_index) + '/'

cli: Configuration from ./config/default.yaml


In [183]:
"""def eval_faiss(emb_dir,
               emb_dummy_dir=None,
               index_type='ivfpq',
               nogpu=False,
               max_train=1e7,
               test_ids='icassp',
               test_seq_len='1 3 5 9 11 19',
               k_probe=20,
               display_interval=5):"""

"""
Segment/sequence-wise audio search experiment and evaluation: implementation based on FAISS.

ex) python eval.py EMB_DIR --index_type ivfpq

EMB_DIR: Directory where {query, db, dummy_db}.mm files are located. The 'raw_score.npy' and 'test_ids.npy' will be also created in the same directory.
"""

"\nSegment/sequence-wise audio search experiment and evaluation: implementation based on FAISS.\n\nex) python eval.py EMB_DIR --index_type ivfpq\n\nEMB_DIR: Directory where {query, db, dummy_db}.mm files are located. The 'raw_score.npy' and 'test_ids.npy' will be also created in the same directory.\n"

In [184]:
test_seq_len = np.asarray(
    list(map(int, test_seq_len.split())))  # '1 3 5' --> [1, 3, 5]
#'11' -> array([11])

In [185]:
emb_dir

'./logs/emb/Checks_test_generate/100/'

In [186]:
# Load items from {query, db, dummy_db}
query, query_shape = load_memmap_data(emb_dir, 'query')
db, db_shape = load_memmap_data(emb_dir, 'db')
if emb_dummy_dir is None:
    emb_dummy_dir = emb_dir
dummy_db, dummy_db_shape = load_memmap_data(emb_dummy_dir, 'dummy_db')

Load 29,500 items from ./logs/emb/Checks_test_generate/100/query.mm.
Load 29,500 items from ./logs/emb/Checks_test_generate/100/db.mm.
Load 53,754,198 items from ./logs/emb/Checks_test_generate/100/dummy_db.mm.


In [187]:
""" ----------------------------------------------------------------------
FAISS index setup

    dummy: 10 items.
    db: 5 items.
    query: 5 items, corresponding to 'db'.

    index.add(dummy_db); index.add(db) # 'dummy_db' first

            |------ dummy_db ------|
    index: [d0, d1, d2,..., d8, d9, d11, d12, d13, d14, d15]
                                    |--------- db ----------|

                                    |--------query ---------|
                                    [q0,  q1,  q2,  q3,  q4]

• The set of ground truth IDs for q[i] will be (i + len(dummy_db))

---------------------------------------------------------------------- """

" ----------------------------------------------------------------------\nFAISS index setup\n\n    dummy: 10 items.\n    db: 5 items.\n    query: 5 items, corresponding to 'db'.\n\n    index.add(dummy_db); index.add(db) # 'dummy_db' first\n\n            |------ dummy_db ------|\n    index: [d0, d1, d2,..., d8, d9, d11, d12, d13, d14, d15]\n                                    |--------- db ----------|\n\n                                    |--------query ---------|\n                                    [q0,  q1,  q2,  q3,  q4]\n\n• The set of ground truth IDs for q[i] will be (i + len(dummy_db))\n\n---------------------------------------------------------------------- "

In [ ]:
# Create and train FAISS index
index = get_index(index_type, dummy_db, dummy_db.shape, (not nogpu), max_train)

# Add items to index
start_time = time.time()

index.add(dummy_db); print(f'{len(dummy_db)} items from dummy DB')
index.add(db); print(f'{len(db)} items from reference DB')

t = time.time() - start_time
print(f'Added total {index.ntotal} items to DB. {t:>4.2f} sec.')

In [ ]:
""" ----------------------------------------------------------------------
We need to prepare a merged {dummy_db + db} memmap:

• Calcuation of sequence-level matching score requires reconstruction of
    vectors from FAISS index.
• Unforunately, current faiss.index.reconstruct_n(id_start, id_stop)
    supports only CPU index.
• We prepare a fake_recon_index thourgh the on-disk method.

---------------------------------------------------------------------- """

In [188]:
# Prepare fake_recon_index
del dummy_db
start_time = time.time()

fake_recon_index, index_shape = load_memmap_data(
    emb_dummy_dir, 'dummy_db', append_extra_length=query_shape[0],
    display=False)
fake_recon_index[dummy_db_shape[0]:dummy_db_shape[0] + query_shape[0], :] = db[:, :]
fake_recon_index.flush()

t = time.time() - start_time
print(f'Created fake_recon_index, total {index_shape[0]} items. {t:>4.2f} sec.')

Created fake_recon_index, total 53783698 items. 0.06 sec.


In [189]:
# Get test_ids
print(f'test_id: \033[93m{test_ids}\033[0m,  ', end='')
if test_ids.lower() == 'all':
    test_ids = np.arange(0, len(query) - max(test_seq_len), 1) # will test all segments in query/db set
elif test_ids.lower() == 'icassp':
    test_ids = np.load(
        glob.glob('./**/test_ids_icassp2021.npy', recursive=True)[0])
elif test_ids.isnumeric():
    test_ids = np.random.permutation(len(query) - max(test_seq_len))[:int(test_ids)]
else:
    test_ids = np.load(test_ids)

n_test = len(test_ids)
gt_ids  = test_ids + dummy_db_shape[0]
print(f'n_test: \033[93m{n_test:n}\033[0m')

test_id: icassp,  n_test: 2000


In [196]:
""" Segement/sequence-level search & evaluation """
# Define metric
top1_exact = np.zeros((n_test, len(test_seq_len))).astype(int) # (n_test, test_seg_len)
top1_near = np.zeros((n_test, len(test_seq_len))).astype(int)
top3_exact = np.zeros((n_test, len(test_seq_len))).astype(int)
top10_exact = np.zeros((n_test, len(test_seq_len))).astype(int)
# top1_song = np.zeros((n_test, len(test_seq_len))).astype(np.int)

In [200]:
scr = curses.initscr()
pt = PrintTable(scr=scr, test_seq_len=test_seq_len,
                row_names=['Top1 exact', 'Top1 near', 'Top3 exact','Top10 exact'])
start_time = time.time()
for ti, test_id in enumerate(test_ids):
    print(f"ti={ti}")
    print(f"test_id={test_id}")
    gt_id = gt_ids[ti]
    print(f"gt_id={gt_id}")
    for si, sl in enumerate(test_seq_len):
        print(f"si={si}")
        print(f"sl={sl}")
        assert test_id <= len(query)
        q = query[test_id:(test_id + sl), :] # shape(q) = (length, dim)
        print(f"q={q}")

        # segment-level top k search for each segment
        _, I = index.search(q, k_probe) # _: distance, I: result IDs matrix

        # offset compensation to get the start IDs of candidate sequences
        for offset in range(len(I)):
            I[offset, :] -= offset

        # unique candidates
        candidates = np.unique(I[np.where(I >= 0)])   # ignore id < 0

        """ Sequence match score """
        _scores = np.zeros(len(candidates))
        for ci, cid in enumerate(candidates):
            _scores[ci] = np.mean(
                np.diag(
                    # np.dot(q, index.reconstruct_n(cid, (cid + l)).T)
                    np.dot(q, fake_recon_index[cid:cid + sl, :].T)
                    )
                )

        """ Evaluate """
        pred_ids = candidates[np.argsort(-_scores)[:10]]
        # pred_id = candidates[np.argmax(_scores)] <-- only top1-hit

        # top1 hit
        top1_exact[ti, si] = int(gt_id == pred_ids[0])
        top1_near[ti, si] = int(
            pred_ids[0] in [gt_id - 1, gt_id, gt_id + 1])
        # top1_song = need song info here...

        # top3, top10 hit
        top3_exact[ti, si] = int(gt_id in pred_ids[:3])
        top10_exact[ti, si] = int(gt_id in pred_ids[:10])


    if (ti != 0) & ((ti % display_interval) == 0):
        avg_search_time = (time.time() - start_time) / display_interval \
            / len(test_seq_len)
        top1_exact_rate = 100. * np.mean(top1_exact[:ti + 1, :], axis=0)
        top1_near_rate = 100. * np.mean(top1_near[:ti + 1, :], axis=0)
        top3_exact_rate = 100. * np.mean(top3_exact[:ti + 1, :], axis=0)
        top10_exact_rate = 100. * np.mean(top10_exact[:ti + 1, :], axis=0)
        # top1_song = 100 * np.mean(tp_song[:ti + 1, :], axis=0)

        pt.update_counter(ti, n_test, avg_search_time * 1000.)
        pt.update_table((top1_exact_rate, top1_near_rate, top3_exact_rate,
                            top10_exact_rate))
        start_time = time.time() # reset stopwatch

ti=0
test_id=14655
gt_id=53768853
si=0
sl=11
q=[[ 0.10253137 -0.00045375  0.0620372  ... -0.11246537 -0.10671374
  -0.05634388]
 [ 0.04767687 -0.07006861  0.08062644 ... -0.19844265 -0.02969167
  -0.00992421]
 [ 0.03126371 -0.05754444  0.11367033 ... -0.15368056  0.01969014
   0.07624688]
 ...
 [-0.05556197 -0.0404008   0.00360346 ... -0.13998988 -0.11149226
   0.001388  ]
 [ 0.05866905 -0.00876279 -0.04984107 ... -0.08355431 -0.0883724
  -0.08032892]
 [ 0.16766731  0.01832582 -0.09314838 ... -0.03284546 -0.08796763
  -0.05063683]]


NameError: name 'index' is not defined

In [ ]:
# Summary
top1_exact_rate = 100. * np.mean(top1_exact, axis=0)
top1_near_rate = 100. * np.mean(top1_near, axis=0)
top3_exact_rate = 100. * np.mean(top3_exact, axis=0)
top10_exact_rate = 100. * np.mean(top10_exact, axis=0)
# top1_song = 100 * np.mean(top1_song[:ti + 1, :], axis=0)

pt.update_counter(ti, n_test, avg_search_time * 1000.)
pt.update_table((top1_exact_rate, top1_near_rate, top3_exact_rate, top10_exact_rate))
pt.close_table() # close table and print summary
del fake_recon_index, query, db
np.save(f'{emb_dir}/raw_score.npy',
        np.concatenate(
            (top1_exact, top1_near, top3_exact, top10_exact), axis=1))
np.save(f'{emb_dir}/test_ids.npy', test_ids)
print(f'Saved test_ids and raw score to {emb_dir}.')

In [ ]:
if nogpu:
    eval_faiss([emb_dir, "--index_type", index_type, "--test_seq_len",
                test_seq_len, "--test_ids", test_ids, "--nogpu"])
else:
    eval_faiss([emb_dir, "--index_type", index_type, "--test_seq_len",
                test_seq_len, "--test_ids", test_ids])

In [ ]:
def evaluate(checkpoint_name, checkpoint_index, config, index_type,
             test_seq_len, test_ids, nogpu):
    """ Search and evalutation.

    ex) python run.py evaluate CHECKPOINT_NAME CHECKPOINT_INDEX

    With options: \b\n

    ex) python run.py evaluate CHECKPOINT_NAME CHEKPOINT_INDEX -i ivfpq -t 3000 --nogpu


    • Currently, the 'evaluate' command does not reference any information other
    than the output log directory from the config file.
    """
    

    if nogpu:
        eval_faiss([emb_dir, "--index_type", index_type, "--test_seq_len",
                    test_seq_len, "--test_ids", test_ids, "--nogpu"])
    else:
        eval_faiss([emb_dir, "--index_type", index_type, "--test_seq_len",
                    test_seq_len, "--test_ids", test_ids])